# **Train New Model with TCGA pretrained model**

## **Module**

In [1]:
from function import *

/home/wmbio/anaconda3/envs/deepdep/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wmbio/anaconda3/envs/deepdep/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wmbio/anaconda3/envs/deepdep/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/wmbio/anaconda3/envs/deepdep/lib/python3

In [2]:
print(os.getcwd())
os.chdir("/home/wmbio/WORK/gitworking/Dependency_prediction/")
print(os.getcwd())

/home/wmbio/WORK/gitworking/Dependency_prediction/prediction/src
/home/wmbio/WORK/gitworking/Dependency_prediction


## **Data Load & Path**

* **Path**

In [3]:
TRAIN_PATH = "/home/wmbio/WORK/gitworking/Dependency_prediction/preprocessing/DATA/2022-08-18//"
PREMODEL_PATH = "prediction/data/premodel_tcga/"
TEMP_PATH = "prediction/train_preprocessing/"
SAVE_PATH = "prediction/custom_model/" + datetime.datetime.now().strftime("%Y-%m%d") + "/"
CHECK_PATH = SAVE_PATH + "checkpoint/"
NPZ_PATH = 'prediction/data/ccl_complete_data_501CCL_1632DepOI_.npz'
Path(CHECK_PATH).mkdir(parents=True, exist_ok=True)

* **Pretrained Model load - TCGA**

In [4]:
premodel_mut = pickle.load(open(PREMODEL_PATH + 'premodel_tcga_mut_1000_100_50.pickle', 'rb'))
premodel_exp = pickle.load(open(PREMODEL_PATH + 'premodel_tcga_exp_500_200_50.pickle', 'rb'))
premodel_cna = pickle.load(open(PREMODEL_PATH + 'premodel_tcga_cna_500_200_50.pickle', 'rb'))
premodel_meth = pickle.load(open(PREMODEL_PATH + 'premodel_tcga_meth_500_200_50.pickle', 'rb'))
print("\n\nDatasets successfully loaded.")



Datasets successfully loaded.


## **Fit & Evaluation Model**

### **(1) Full Model 4-Omics**

In [ ]:
# data load & train_test split
data_exp, data_mut, data_cna, data_meth, data_fprint, data_dep = trainset_load(npz_path=NPZ_PATH,
                                                                              temp_path=TEMP_PATH, train_path=TRAIN_PATH)
id_train, id_test = train_test_index(index_data=data_mut, num_depoi=1632)

with tf.device('/cpu:0'):
    K.clear_session()
    model = full_model(data_mut=data_mut, data_exp=data_exp, data_cna=data_cna,data_meth=data_meth,
                      data_fprint=data_fprint, data_dep=data_dep, id_train=id_train, id_test=id_test,
                      premodel_mut=premodel_mut, premodel_exp=premodel_exp, premodel_cna=premodel_cna,
                      premodel_meth=premodel_meth, save_path = CHECK_PATH)



Training/validation on 612656 samples (472 CCLs x 1298 DepOIs) and testing on 203786 samples (157 CCLs x 1298 DepOIs).




In [ ]:
model_train_vis(history=model[1], save_path=SAVE_PATH, filename="Full")

In [ ]:
# SAVE_NAME = "model_full"
SAVE_NAME = "model_cna-exp-meth-mut"
model[0].save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(2) Expression-Mutation-CNA Model 3-Omics**

In [ ]:
data_exp, data_mut, data_cna, data_meth, data_fprint, data_dep = trainset_load(npz_path=NPZ_PATH,
                                                                               temp_path=TEMP_PATH, train_path=TRAIN_PATH, 
                                                                               meth=False)
id_train, id_test = train_test_index(data_mut)

with tf.device('/cpu:0'):
    K.clear_session()
    model = mut_exp_cna_model(data_mut=data_mut, data_exp=data_exp, data_cna=data_cna,
                      data_fprint=data_fprint, data_dep=data_dep, id_train=id_train, id_test=id_test,
                      premodel_mut=premodel_mut, premodel_exp=premodel_exp, premodel_cna=premodel_cna,
                      save_path = CHECK_PATH)

In [ ]:
model_train_vis(history=model[1], save_path=SAVE_PATH, filename="Mut-Exp-CNA")

In [ ]:
# SAVE_NAME = "model_mut_exp_cna"
SAVE_NAME = "model_cna-exp-mut"
model[0].save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nMut_Exp_CNA-DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(3) Expression-Mutation-Meth Model 3-Omics**

In [ ]:
data_exp, data_mut, data_cna, data_meth, data_fprint, data_dep = trainset_load(npz_path=NPZ_PATH,
                                                                               temp_path=TEMP_PATH, train_path=TRAIN_PATH, 
                                                                               cna=False)
id_train, id_test = train_test_index(data_mut)

with tf.device('/cpu:0'):
    K.clear_session()
    model = mut_exp_meth_model(data_mut=data_mut, data_exp=data_exp, data_meth=data_meth,
                      data_fprint=data_fprint, data_dep=data_dep, id_train=id_train, id_test=id_test,
                      premodel_mut=premodel_mut, premodel_exp=premodel_exp, premodel_meth=premodel_meth,
                      save_path = CHECK_PATH)

In [ ]:
model_train_vis(history=model[1], save_path=SAVE_PATH, filename="Mut-Exp-Meth")

In [ ]:
# SAVE_NAME = "model_mut_exp_meth"
SAVE_NAME = "model_exp-meth-mut"
model[0].save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nMut_Exp_Meth-DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(4) Mutation-CNA-Meth Model 3-Omics**

In [ ]:
data_exp, data_mut, data_cna, data_meth, data_fprint, data_dep = trainset_load(npz_path=NPZ_PATH,
                                                                               temp_path=TEMP_PATH, train_path=TRAIN_PATH,
                                                                               exp=False)
id_train, id_test = train_test_index(data_mut)

with tf.device('/cpu:0'):
    K.clear_session()
    model = mut_cna_meth_model(data_mut=data_mut, data_cna=data_cna, data_meth=data_meth,
                      data_fprint=data_fprint, data_dep=data_dep, id_train=id_train, id_test=id_test,
                      premodel_mut=premodel_mut, premodel_cna=premodel_cna, premodel_meth=premodel_meth,
                      save_path = CHECK_PATH)

In [ ]:
model_train_vis(history=model[1], save_path=SAVE_PATH, filename="Mut-CNA-Meth")

In [ ]:
# SAVE_NAME = "model_mut_cna_meth"
SAVE_NAME = "model_cna-meth-mut"
model[0].save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nMut_CNA_Meth-DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(5) Expression-Mutation Model**

In [ ]:
data_exp, data_mut, data_cna, data_meth, data_fprint, data_dep = trainset_load(npz_path=NPZ_PATH,
                                                                               temp_path=TEMP_PATH, train_path=TRAIN_PATH, 
                                                                               cna=False, meth=False)
id_train, id_test = train_test_index(data_mut)

with tf.device('/cpu:0'):
    K.clear_session()
    model = mut_exp_model(data_mut=data_mut, data_exp=data_exp, 
                      data_fprint=data_fprint, data_dep=data_dep, id_train=id_train, id_test=id_test,
                      premodel_mut=premodel_mut, premodel_exp=premodel_exp,
                      save_path = CHECK_PATH)

In [ ]:
model_train_vis(history=model[1], save_path=SAVE_PATH, filename="Mut-Exp")

In [ ]:
# SAVE_NAME = "model_mut_exp"
SAVE_NAME = "model_exp-mut"
model[0].save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nMut_CNA-DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(6) Expression Model**

In [ ]:
data_exp, data_mut, data_cna, data_meth, data_fprint, data_dep = trainset_load(npz_path=NPZ_PATH,
                                                                               temp_path=TEMP_PATH, train_path=TRAIN_PATH,
                                                                               mut=False, cna=False, meth=False)
id_train, id_test = train_test_index(data_exp)

with tf.device('/cpu:0'):
    K.clear_session()
    model = exp_model(data_exp=data_exp, data_fprint=data_fprint, data_dep=data_dep, 
                      id_train=id_train, id_test=id_test, premodel_exp=premodel_exp,
                      save_path = CHECK_PATH)

In [ ]:
model_train_vis(history=model[1], save_path=SAVE_PATH, filename="Exp")

In [ ]:
SAVE_NAME = "model_exp"
model[0].save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\Exp-DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(7) Mutation Model**

In [ ]:
data_exp, data_mut, data_cna, data_meth, data_fprint, data_dep = trainset_load(npz_path=NPZ_PATH,
                                                                               temp_path=TEMP_PATH, train_path=TRAIN_PATH,
                                                                               exp=False, cna=False, meth=False)
id_train, id_test = train_test_index(data_mut)

with tf.device('/cpu:0'):
    K.clear_session()
    model = mut_model(data_mut=data_mut, data_fprint=data_fprint, data_dep=data_dep, 
                      id_train=id_train, id_test=id_test, premodel_mut=premodel_mut,
                      save_path = CHECK_PATH)

In [ ]:
model_train_vis(history=model[1], save_path=SAVE_PATH, filename="Mut")

In [ ]:
SAVE_NAME = "model_mut"
model[0].save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(8) CNA Model**

In [ ]:
data_exp, data_mut, data_cna, data_meth, data_fprint, data_dep = trainset_load(npz_path=NPZ_PATH,
                                                                               temp_path=TEMP_PATH, train_path=TRAIN_PATH, 
                                                                               mut=False, exp=False, meth=False)
id_train, id_test = train_test_index(data_cna)

with tf.device('/cpu:0'):
    K.clear_session()
    model = cna_model(data_cna=data_cna, data_fprint=data_fprint, data_dep=data_dep, 
                      id_train=id_train, id_test=id_test, premodel_cna=premodel_cna,
                      save_path = CHECK_PATH)

In [ ]:
model_train_vis(history=model[1], save_path=SAVE_PATH, filename="CNA")

In [ ]:
SAVE_NAME = "model_cna"
model[0].save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\CNA DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(9) Meth Model**

In [ ]:
data_exp, data_mut, data_cna, data_meth, data_fprint, data_dep = trainset_load(npz_path=NPZ_PATH,
                                                                               temp_path=TEMP_PATH, train_path=TRAIN_PATH, 
                                                                               mut=False, cna=False, exp=False)
id_train, id_test = train_test_index(data_meth)

with tf.device('/cpu:0'):
    K.clear_session()
    model = meth_model(data_meth=data_meth, data_fprint=data_fprint, data_dep=data_dep, 
                      id_train=id_train, id_test=id_test, premodel_meth=premodel_meth,
                      save_path = CHECK_PATH)

In [ ]:
model_train_vis(history=model[1], save_path=SAVE_PATH, filename="Meth")

In [ ]:
SAVE_NAME = "model_meth"
model[0].save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nMeth DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))